In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.image as mpimg

import torch
import torch.nn as nn
import torch.optim as optim 

import torchvision
from torch.utils.data import DataLoader, Dataset
import torch.utils.data as utils
from torchvision import transforms

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")


In [2]:
data_dir = '/home/hasan/Data Set/aerial-cactus-identification'

train_dir = data_dir + '/train/'
test_dir = data_dir + '/test/'


In [3]:
labels = pd.read_csv("/home/hasan/Data Set/aerial-cactus-identification/train.csv")
labels.head()

,id,has_cactus
0,0004be2cfeaba1c0361d39e2b000257b.jpg,1
1,000c8a36845c0208e833c79c1bffedd1.jpg,1
2,000d1e9a533f62e55c289303b072733d.jpg,1
3,0011485b40695e9138e92d0b3fb55128.jpg,1
4,0014d7a11e90b62848904c1418fc8cf2.jpg,1


In [4]:
class ImageData(Dataset):
    def __init__(self, df, data_dir, transform):
        super().__init__()
        self.df = df
        self.data_dir = data_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):       
        img_name = self.df.id[index]
        label = self.df.has_cactus[index]
        
        img_path = os.path.join(self.data_dir, img_name)
        image = mpimg.imread(img_path)
        image = self.transform(image)
        return image, label

In [5]:
data_transf = transforms.Compose([transforms.ToPILImage(), transforms.ToTensor()])

train_data = ImageData(df = labels, data_dir = train_dir, transform = data_transf)
train_loader = DataLoader(dataset = train_data, batch_size = 64)


In [33]:
from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_name('efficientnet-b1')

In [34]:
# Unfreeze model weights
for param in model.parameters():
    param.requires_grad = True

In [35]:
# Adding output layers 
num_ftrs = model._fc.in_features
model._fc = nn.Linear(num_ftrs, 1)



In [36]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)

In [37]:
optimizer = optim.Adam(model.parameters())
loss_func = nn.BCELoss()

In [ ]:
# Train model
loss_log = []

for epoch in range(5):    
    model.train()    
    for ii, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        target = target.float()                

        optimizer.zero_grad()
        output = model(data)                
    
        m = nn.Sigmoid()
        loss = loss_func(m(output), target)
        loss.backward()

        optimizer.step()  
        
        if ii % 1000 == 0:
            loss_log.append(loss.item())
       
    print('Epoch: {} - Loss: {:.6f}'.format(epoch + 1, loss.item()))

In [23]:
submit = pd.read_csv('/home/hasan/Data Set/aerial-cactus-identification/sample_submission.csv')

test_data = ImageData(df = submit, data_dir = test_dir, transform = data_transf)
test_loader = DataLoader(dataset = test_data, shuffle=False)


In [ ]:
predict = []
model.eval()
for i, (data, _) in enumerate(test_loader):
    data = data.to(device)
    output = model(data)    

    pred = torch.sigmoid(output)
    predicted_vals = pred > 0.5
    predict.append(int(predicted_vals))
    
submit['has_cactus'] = predict
submit.to_csv('submission.csv', index=False)

In [27]:
submit.head()

,id,has_cactus
0,000940378805c44108d287872b2f04ce.jpg,0.5
1,0017242f54ececa4512b4d7937d1e21e.jpg,0.5
2,001ee6d8564003107853118ab87df407.jpg,0.5
3,002e175c3c1e060769475f52182583d0.jpg,0.5
4,0036e44a7e8f7218e9bc7bf8137e4943.jpg,0.5
